<a href="https://colab.research.google.com/github/Aingon14/BSC_DPDM23/blob/main/Group_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd
import numpy as np
import os

In [80]:
data_truck = pd.read_excel('/content/drive/MyDrive/BSC_DPDM23/grop projcet/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx')

# Data truck 2560

In [81]:
data_dict_truck = pd.read_excel('/content/drive/MyDrive/BSC_DPDM23/grop projcet/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx', sheet_name = 0)
data_dict_truck.head()

,Column,Description,Data Type,ชื่อตัวแปร,ค่าที่เป็นไปได้
0,Date,วันเดือนปีที่เกิดเหตุ,number,NaN,NaN
1,Time,ช่วงเวลาที่เกิดเหตุ,number,16.01 - 20.00 น.,1.0
2,NaN,NaN,NaN,20.01 - 00.00 น.,2.0
3,NaN,NaN,NaN,00.01 - 04.00 น.,3.0
4,NaN,NaN,NaN,04.01 - 08.00 น.,4.0


In [82]:
data_2560 = pd.read_excel('/content/drive/MyDrive/BSC_DPDM23/grop projcet/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx', sheet_name = 1)
data_2560.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,02/01/2560,11:16:00,ขอนแก่น,3,รถอื่นๆ,รถบรรทุก,หลับใน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04/01/2560,07:30:00,แพร่,2,ไม่มีคู่กรณี,รถบรรทุก,หลับใน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04/01/2560,20:10:00,สระบุรี,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04/01/2560,21:10:00,เพชรบุรี,2,รถจักรยานยนต์,รถจักรยานยนต์,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05/01/2560,07:30:00,เชียงราย,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับรถเร็ว,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
data60_col = data_2560[['Date','Time','Province','Loss','Party','Cause','Main cause of accidents']]
data60_col.head()  #เลือกเฉพาะคอลัมน์ที่ต้องการ

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,ขอนแก่น,3,รถอื่นๆ,รถบรรทุก,หลับใน
1,04/01/2560,07:30:00,แพร่,2,ไม่มีคู่กรณี,รถบรรทุก,หลับใน
2,04/01/2560,20:10:00,สระบุรี,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน
3,04/01/2560,21:10:00,เพชรบุรี,2,รถจักรยานยนต์,รถจักรยานยนต์,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน
4,05/01/2560,07:30:00,เชียงราย,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับรถเร็ว


In [84]:
#เปลี่ยนชื่อจังหวัดเป็นภาษาอังกฤษ
data_truck_60 = data60_col
datamain60 = pd.DataFrame(data_truck_60)
datamain60['Province'] = datamain60['Province'].apply(lambda x: x.strip())
datamain60['Province'] = datamain60['Province'].replace({
    'เชียงใหม่': 'Chiang Mai',
    'เชียงราย': 'Chiang Rai',
    'ลำปาง': 'Lampang',
    'ลำพูน': 'Lamphun',
    'แม่ฮ่องสอน': 'Mae Hong Son',
    'น่าน': 'Nan',
    'พะเยา': 'Phayao',
    'แพร่': 'Phrae',
    'อุตรดิตถ์': 'Uttaradit',
    'กาฬสินธุ์': 'Kalasin',
    'ขอนแก่น': 'Khon Kaen',
    'ชัยภูมิ': 'Chaiyaphum',
    'นครพนม': 'Nakhon Phanom',
    'นครราชสีมา': 'Nakhon Ratchasima',
    'บึงกาฬ': 'Bueng Kan',
    'บุรีรัมย์': 'Buriram',
    'มหาสารคาม': 'Maha Sarakham',
    'มุกดาหาร': 'Mukdahan',
    'ยโสธร': 'Yasothon',
    'ร้อยเอ็ด': 'Roi Et',
    'เลย': 'Loei',
    'สกลนคร': 'Sakon Nakhon',
    'สุรินทร์': 'Surin',
    'ศรีสะเกษ': 'Sisaket',
    'หนองคาย': 'Nong Khai',
    'หนองบัวลำภู': 'Nong Bua Lamphu',
    'อุดรธานี': 'Udon Thani',
    'อุบลราชธานี': 'Ubon Ratchathani',
    'อำนาจเจริญ': 'Amnat Charoen',
    'กรุงเทพมหานคร': 'Bangkok',
    'กำแพงเพชร': 'Kamphaeng Phet',
    'ชัยนาท': 'Chai Nat',
    'นครนายก': 'Nakhon Nayok',
    'นครปฐม': 'Nakhon Pathom',
    'นครสวรรค์': 'Nakhon Sawan',
    'นนทบุรี': 'Nonthaburi',
    'ปทุมธานี': 'Pathum Thani',
    'พระนครศรีอยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พิจิตร': 'Phichit',
    'พิษณุโลก': 'Phitsanulok',
    'เพชรบูรณ์': 'Phetchabun',
    'ลพบุรี': 'Lopburi',
    'สมุทรปราการ': 'Samut Prakan',
    'สมุทรสงคราม': 'Samut Songkhram',
    'สมุทรสาคร': 'Samut Sakhon',
    'สิงห์บุรี': 'Sing Buri',
    'สุโขทัย': 'Sukhothai',
    'สุพรรณบุรี': 'Suphan Buri',
    'สระบุรี': 'Saraburi',
    'อ่างทอง': 'Ang Thong',
    'อุทัยธานี': 'Uthai Thani',
    'จันทบุรี': 'Chanthaburi',
    'ฉะเชิงเทรา': 'Chachoengsao',
    'ชลบุรี': 'Chonburi',
    'ตราด': 'Trat',
    'ปราจีนบุรี': 'Prachinburi',
    'ระยอง': 'Rayong',
    'สระแก้ว': 'Sa Kaeo',
    'กาญจนบุรี': 'Kanchanaburi',
    'ตาก': 'Tak',
    'ประจวบคีรีขันธ์': 'Prachuap Khiri Khan',
    'เพชรบุรี': 'Phetchaburi',
    'ราชบุรี': 'Ratchaburi',
    'กระบี่': 'Krabi',
    'ชุมพร': 'Chumphon',
    'ตรัง': 'Trang',
    'นครศรีธรรมราช': 'Nakhon Si Thammarat',
    'นราธิวาส': 'Narathiwat',
    'ปัตตานี': 'Pattani',
    'พังงา': 'Phang Nga',
    'พัทลุง': 'Phatthalung',
    'ภูเก็ต': 'Phuket',
    'ระนอง': 'Ranong',
    'สตูล': 'Satun',
    'สงขลา': 'Songkhla',
    'สุราษฎร์ธานี': 'Surat Thani',
    'ยะลา': 'Yala'})

In [69]:
data_truck_60.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,Khon Kaen,3,รถอื่นๆ,รถบรรทุก,หลับใน
1,04/01/2560,07:30:00,Phrae,2,ไม่มีคู่กรณี,รถบรรทุก,หลับใน
2,04/01/2560,20:10:00,Saraburi,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน
3,04/01/2560,21:10:00,Phetchaburi,2,รถจักรยานยนต์,รถจักรยานยนต์,ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน
4,05/01/2560,07:30:00,Chiang Rai,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,ขับรถเร็ว


In [85]:
#เปลี่ยนสาเหตุเป็นตัวเลขที่กำหนดไว้
data_truck_600 = data_truck_60
datamain60 = pd.DataFrame(data_truck_600)
datamain60['Main cause of accidents'] = datamain60['Main cause of accidents'].apply(lambda x: x.strip())
datamain60['Main cause of accidents'] = datamain60['Main cause of accidents'].replace({'ขับรถเร็ว':'M1',
'ขับรถตัดหน้าในระยะกระชั้นชิด':'M2',
'ขับตามหลังในระยะกระชั้นชิด':'M3',
'หลับใน':'M4',
' หลับใน':'M4',
'ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน':	'M5',
'ไม่ชำนาญเส้นทาง':	'M6',
'ขาดทักษะที่ถูกต้องในการขับรถ':	'M7',
'ขับล้ำเข้าไปในช่องทางจราจรอื่น':	'M8',
'ขับล้ำเข้าไปในช่องทางฝั่งตรงข้าม':	'M9',
'ฝ่าฝืนเครื่องหมาย/สัญญาณจราจร':	'M10',
'เปลี่ยนช่องทางกระทันหัน':	'M11',
'เมาสุรา':	'M12',
'ใช้สารเสพติดออกฤทธิ์ต่อจิตประสาท':	'M13',
'ขาดสมาธิขณะขับรถ':	'M14',
'ขับย้อนศร':	'M15',
'จอดรถไหล่ทาง':	'M16',
'จอดในที่ห้ามจอด/จอดไหล่ทางล้ำ/จอดไม่ให้เครื่องหมายหรือสัญญาณ':	'M17',
'ทำกิจกรรมอื่นในขณะขับรถ (ก้มหยิบของ,โทรศัพท์)':	'M18',
'สภาพยางชำรุด':	'M19',
'ระบบเบรกชำรุด':	'M20',
'ระบบคันส่งคันชักชำรุด':	'M21',
'ระบบล้อและเพลาชำรุด':	'M22',
'ระบบไฟฟ้าชำรุด':	'M23',
'ระบบพวงมาลัยชำรุด':	'M24',
'อุปกรณ์ส่วนควบอื่นๆชำรุด':	'M25',
'สภาพถนนชำรุด':	'M26',
'ฝนตกถนนลื่น':	'M27',
'ถนนไม่มีไฟฟ้าแสงสว่าง':	'M28',
'ถนนมีสิ่งกีดขวางจราจร':	'M29',
'ประมาทร่วม':	'M30',
'รถอีกคันกระเด็นมาถูกโดยบังเอิญ':	'M31',
'ปัญหาสุขภาพผู้ขับรถ':	'M32',
'ไม่ระบุ':	'M33',
'คนเดินตัดหน้ารถ'	:'M34' })

In [86]:
data_truck_600.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,Khon Kaen,3,รถอื่นๆ,รถบรรทุก,M4
1,04/01/2560,07:30:00,Phrae,2,ไม่มีคู่กรณี,รถบรรทุก,M4
2,04/01/2560,20:10:00,Saraburi,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,M5
3,04/01/2560,21:10:00,Phetchaburi,2,รถจักรยานยนต์,รถจักรยานยนต์,M5
4,05/01/2560,07:30:00,Chiang Rai,2,รถยนต์นั่งส่วนบุคคลเกิน7คน,รถยนต์นั่งส่วนบุคคลเกิน7คน,M1


In [87]:
# เปลี่ยนตัวแปรคู่กรณี (Party) เป็นตัวเลขที่กำหนดไว้
data_truck_6000 = data_truck_600
datapar60 = pd.DataFrame(data_truck_6000)
datapar60['Party'] = datapar60['Party'].apply(lambda x: x.strip())
datapar60['Party'] = datapar60['Party'].replace({'รถยนต์บรรทุกส่วนบุคคล (ปิคอัพ)':	'P1',
'รถยนต์นั่งส่วนบุคคลไม่เกิน7คน':	'P2',
'รถยนต์นั่งส่วนบุคคลเกิน7คน':	'P3',
'รถบรรทุก':	'P4',
'รถโดยสาร':	'P5',
'รถจักรยานยนต์':	'P6',
'รถจักรยาน':	'P7',
'รถอื่นๆ':	'P8',
'คู่กรณีมากกว่า 1 คัน':	'P9',
'เสาไฟฟ้า':	'P10',
'ป้าย':'P11',
'ต้นไม้':'P12',
'ไม่มีคู่กรณี': 'P13',
'คนเดินเท้า':	'P14',
'ไม่ระบุ':	'P15' })

In [88]:
data_truck_6000.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,Khon Kaen,3,P8,รถบรรทุก,M4
1,04/01/2560,07:30:00,Phrae,2,P13,รถบรรทุก,M4
2,04/01/2560,20:10:00,Saraburi,2,P3,รถยนต์นั่งส่วนบุคคลเกิน7คน,M5
3,04/01/2560,21:10:00,Phetchaburi,2,P6,รถจักรยานยนต์,M5
4,05/01/2560,07:30:00,Chiang Rai,2,P3,รถยนต์นั่งส่วนบุคคลเกิน7คน,M1


In [90]:
# เปลี่ยนตัวแปรรถต้นเหตุ (Cause) เป็นตัวเลขที่กำหนดไว้
data_truck_60000 = data_truck_6000
datapar60 = pd.DataFrame(data_truck_60000)
datapar60['Cause'] = datapar60['Cause'].apply(lambda x: x.strip())
datapar60['Cause'] = datapar60['Cause'].replace({'รถยนต์บรรทุกส่วนบุคคล (ปิคอัพ)':	'C1',
'รถยนต์นั่งส่วนบุคคลไม่เกิน7คน':	'C2',
'รถยนต์นั่งส่วนบุคคลเกิน7คน':	'C3',
'รถบรรทุก':	'C4',
'รถโดยสาร':	'C5',
'รถจักรยานยนต์':	'C6',
'รถจักรยาน':	'C7',
'รถอื่นๆ':	'C8',
'คู่กรณีมากกว่า 1 คัน':	'C9',
'เสาไฟฟ้า':	'C10',
'ป้าย':'C11',
'ต้นไม้':'C12',
'ไม่มีคู่กรณี': 'C13',
'คนเดินเท้า':	'C14',
'ไม่ระบุ':	'C15'})

In [91]:
data_truck_60000.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,11:16:00,Khon Kaen,3,P8,C4,M4
1,04/01/2560,07:30:00,Phrae,2,P13,C4,M4
2,04/01/2560,20:10:00,Saraburi,2,P3,C3,M5
3,04/01/2560,21:10:00,Phetchaburi,2,P6,C6,M5
4,05/01/2560,07:30:00,Chiang Rai,2,P3,C3,M1


# Data truck 2561

In [23]:
data_2561 = pd.read_excel('/content/drive/MyDrive/BSC_DPDM23/grop projcet/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx', sheet_name = 2)
data_2561.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,สมุทรสาคร,1,6,1,16
1,01/01/2561,2.0,ระยอง,1,6,1,16
2,02/01/2561,5.0,สมุทรสาคร,3,1,1,19
3,03/01/2561,6.0,สุราษฎร์ธานี,2,9,1,6
4,03/01/2561,1.0,ระยอง,1,6,6,1


In [24]:
#เปลี่ยนชื่อจังหวัดเป็นภาษาอังกฤษ
data_truck_61 = data_2561
datamain61 = pd.DataFrame(data_truck_61)
datamain61['Province'] = datamain61['Province'].apply(lambda x: x.strip())
datamain61['Province'] = datamain61['Province'].replace({
    'เชียงใหม่': 'Chiang Mai',
    'เชียงราย': 'Chiang Rai',
    'ลำปาง': 'Lampang',
    'ลำพูน': 'Lamphun',
    'แม่ฮ่องสอน': 'Mae Hong Son',
    'น่าน': 'Nan',
    'พะเยา': 'Phayao',
    'แพร่': 'Phrae',
    'อุตรดิตถ์': 'Uttaradit',
    'กาฬสินธุ์': 'Kalasin',
    'ขอนแก่น': 'Khon Kaen',
    'ชัยภูมิ': 'Chaiyaphum',
    'นครพนม': 'Nakhon Phanom',
    'นครราชสีมา': 'Nakhon Ratchasima',
    'บึงกาฬ': 'Bueng Kan',
    'บุรีรัมย์': 'Buriram',
    'มหาสารคาม': 'Maha Sarakham',
    'มุกดาหาร': 'Mukdahan',
    'ยโสธร': 'Yasothon',
    'ร้อยเอ็ด': 'Roi Et',
    'เลย': 'Loei',
    'สกลนคร': 'Sakon Nakhon',
    'สุรินทร์': 'Surin',
    'ศรีสะเกษ': 'Sisaket',
    'หนองคาย': 'Nong Khai',
    'หนองบัวลำภู': 'Nong Bua Lamphu',
    'อุดรธานี': 'Udon Thani',
    'อุบลราชธานี': 'Ubon Ratchathani',
    'อำนาจเจริญ': 'Amnat Charoen',
    'กรุงเทพมหานคร': 'Bangkok',
    'กำแพงเพชร': 'Kamphaeng Phet',
    'ชัยนาท': 'Chai Nat',
    'นครนายก': 'Nakhon Nayok',
    'นครปฐม': 'Nakhon Pathom',
    'นครสวรรค์': 'Nakhon Sawan',
    'นนทบุรี': 'Nonthaburi',
    'ปทุมธานี': 'Pathum Thani',
    'พระนครศรีอยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พิจิตร': 'Phichit',
    'พิษณุโลก': 'Phitsanulok',
    'เพชรบูรณ์': 'Phetchabun',
    'ลพบุรี': 'Lopburi',
    'สมุทรปราการ': 'Samut Prakan',
    'สมุทรสงคราม': 'Samut Songkhram',
    'สมุทรสาคร': 'Samut Sakhon',
    'สิงห์บุรี': 'Sing Buri',
    'สุโขทัย': 'Sukhothai',
    'สุพรรณบุรี': 'Suphan Buri',
    'สระบุรี': 'Saraburi',
    'อ่างทอง': 'Ang Thong',
    'อุทัยธานี': 'Uthai Thani',
    'จันทบุรี': 'Chanthaburi',
    'ฉะเชิงเทรา': 'Chachoengsao',
    'ชลบุรี': 'Chonburi',
    'ตราด': 'Trat',
    'ปราจีนบุรี': 'Prachinburi',
    'ระยอง': 'Rayong',
    'สระแก้ว': 'Sa Kaeo',
    'กาญจนบุรี': 'Kanchanaburi',
    'ตาก': 'Tak',
    'ประจวบคีรีขันธ์': 'Prachuap Khiri Khan',
    'เพชรบุรี': 'Phetchaburi',
    'ราชบุรี': 'Ratchaburi',
    'กระบี่': 'Krabi',
    'ชุมพร': 'Chumphon',
    'ตรัง': 'Trang',
    'นครศรีธรรมราช': 'Nakhon Si Thammarat',
    'นราธิวาส': 'Narathiwat',
    'ปัตตานี': 'Pattani',
    'พังงา': 'Phang Nga',
    'พัทลุง': 'Phatthalung',
    'ภูเก็ต': 'Phuket',
    'ระนอง': 'Ranong',
    'สตูล': 'Satun',
    'สงขลา': 'Songkhla',
    'สุราษฎร์ธานี': 'Surat Thani',
    'ยะลา': 'Yala'})


In [25]:
data_truck_61

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,Samut Sakhon,1,6,1,16
1,01/01/2561,2.0,Rayong,1,6,1,16
2,02/01/2561,5.0,Samut Sakhon,3,1,1,19
3,03/01/2561,6.0,Surat Thani,2,9,1,6
4,03/01/2561,1.0,Rayong,1,6,6,1
...,...,...,...,...,...,...,...
853,29/12/2561,5.0,Phra Nakhon Si Ayutthaya,4,13,1,4
854,29/12/2561,5.0,Chanthaburi,2,9,1,3
855,29/12/2561,5.0,Lamphun,2,2,1,27
856,29/12/2561,6.0,Chachoengsao,3,1,1,4


In [123]:
data_truck_611 = data_truck_61
datapar61 = pd.DataFrame(data_truck_611)
datapar61['Party'] = datapar61['Party'].apply(lambda x: x.strip())
datapar61['Party'] = datapar61['Party'].replace({'1':	'P1',
'2':	'P2',
'3':	'P3',
'4':	'P4',
'5':	'P5',
'6':	'P6',
'7':	'P7',
'8':	'P8',
'9':	'P9',
'10':	'P10',
'11':'P11',
'12':'P12',
'13': 'P13',
'14':	'P14',
'15':	'P15' })

AttributeError: 'int' object has no attribute 'strip'

In [121]:
data_truck_611 = data_truck_61
datapar61 = pd.DataFrame(data_truck_611)
datapar61['Cause'] = datapar61['Cause'].apply(lambda x: x.strip())
datapar61['Cause'] = datapar61['Cause'].replace({1:	'C1',
2:	'C2',
3:	'C3',
4:	'C4',
5:	'C5',
6:	'C6',
7:	'C7',
8:	'C8',
9:	'C9',
10:	'C10',
11:'C11',
12:'C12',
13: 'C13',
14:	'C14',
15:	'C15'})


AttributeError: 'function' object has no attribute 'strip'

In [105]:
data_truck_611

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,Samut Sakhon,1,6,<bound method Series.apply of 0 1\n1 ...,16
1,01/01/2561,2.0,Rayong,1,6,<bound method Series.apply of 0 1\n1 ...,16
2,02/01/2561,5.0,Samut Sakhon,3,1,<bound method Series.apply of 0 1\n1 ...,19
3,03/01/2561,6.0,Surat Thani,2,9,<bound method Series.apply of 0 1\n1 ...,6
4,03/01/2561,1.0,Rayong,1,6,<bound method Series.apply of 0 1\n1 ...,1
...,...,...,...,...,...,...,...
853,29/12/2561,5.0,Phra Nakhon Si Ayutthaya,4,13,<bound method Series.apply of 0 1\n1 ...,4
854,29/12/2561,5.0,Chanthaburi,2,9,<bound method Series.apply of 0 1\n1 ...,3
855,29/12/2561,5.0,Lamphun,2,2,<bound method Series.apply of 0 1\n1 ...,27
856,29/12/2561,6.0,Chachoengsao,3,1,<bound method Series.apply of 0 1\n1 ...,4


# Data truck 2562

In [26]:
data_2562 = pd.read_excel('/content/drive/MyDrive/BSC_DPDM23/grop projcet/ข้อมูลอุบัติเหตุรถบรรทุก (5).xlsx', sheet_name = 3)
data_2562.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,สิงห์บุรี,1,รถจักรยานยนต์,รถบรรทุก,จอดรถไหล่ทาง
1,01/01/2562,05:00:00,สุพรรณบุรี,2,รถโดยสาร,รถบรรทุก,ขับย้อนศร
2,01/01/2562,12:50:00,พระนครศรีอยุธยา,4,คู่กรณีมากกว่า 1 คัน,รถบรรทุก,ขับตามหลังในระยะกระชั้นชิด
3,02/01/2562,13:30:00,เลย,3,รถยนต์นั่งส่วนบุคคลไม่เกิน7คน,รถบรรทุก,ขับรถตัดหน้าในระยะกระชั้นชิด
4,02/01/2562,17:00:00,ระนอง,1,รถจักรยานยนต์,รถจักรยานยนต์,ขับล้ำเข้าไปในช่องทางจราจรอื่น


In [27]:
#เปลี่ยนชื่อจังหวัดเป็นภาษาอังกฤษ
data_truck_62 = data_2562
datamain62 = pd.DataFrame(data_truck_62)
datamain62['Province'] = datamain62['Province'].apply(lambda x: x.strip())
datamain62['Province'] = datamain62['Province'].replace({
    'เชียงใหม่': 'Chiang Mai',
    'เชียงราย': 'Chiang Rai',
    'ลำปาง': 'Lampang',
    'ลำพูน': 'Lamphun',
    'แม่ฮ่องสอน': 'Mae Hong Son',
    'น่าน': 'Nan',
    'พะเยา': 'Phayao',
    'แพร่': 'Phrae',
    'อุตรดิตถ์': 'Uttaradit',
    'กาฬสินธุ์': 'Kalasin',
    'ขอนแก่น': 'Khon Kaen',
    'ชัยภูมิ': 'Chaiyaphum',
    'นครพนม': 'Nakhon Phanom',
    'นครราชสีมา': 'Nakhon Ratchasima',
    'บึงกาฬ': 'Bueng Kan',
    'บุรีรัมย์': 'Buriram',
    'มหาสารคาม': 'Maha Sarakham',
    'มุกดาหาร': 'Mukdahan',
    'ยโสธร': 'Yasothon',
    'ร้อยเอ็ด': 'Roi Et',
    'เลย': 'Loei',
    'สกลนคร': 'Sakon Nakhon',
    'สุรินทร์': 'Surin',
    'ศรีสะเกษ': 'Sisaket',
    'หนองคาย': 'Nong Khai',
    'หนองบัวลำภู': 'Nong Bua Lamphu',
    'อุดรธานี': 'Udon Thani',
    'อุบลราชธานี': 'Ubon Ratchathani',
    'อำนาจเจริญ': 'Amnat Charoen',
    'กรุงเทพมหานคร': 'Bangkok',
    'กำแพงเพชร': 'Kamphaeng Phet',
    'ชัยนาท': 'Chai Nat',
    'นครนายก': 'Nakhon Nayok',
    'นครปฐม': 'Nakhon Pathom',
    'นครสวรรค์': 'Nakhon Sawan',
    'นนทบุรี': 'Nonthaburi',
    'ปทุมธานี': 'Pathum Thani',
    'พระนครศรีอยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พิจิตร': 'Phichit',
    'พิษณุโลก': 'Phitsanulok',
    'เพชรบูรณ์': 'Phetchabun',
    'ลพบุรี': 'Lopburi',
    'สมุทรปราการ': 'Samut Prakan',
    'สมุทรสงคราม': 'Samut Songkhram',
    'สมุทรสาคร': 'Samut Sakhon',
    'สิงห์บุรี': 'Sing Buri',
    'สุโขทัย': 'Sukhothai',
    'สุพรรณบุรี': 'Suphan Buri',
    'สระบุรี': 'Saraburi',
    'อ่างทอง': 'Ang Thong',
    'อุทัยธานี': 'Uthai Thani',
    'จันทบุรี': 'Chanthaburi',
    'ฉะเชิงเทรา': 'Chachoengsao',
    'ชลบุรี': 'Chonburi',
    'ตราด': 'Trat',
    'ปราจีนบุรี': 'Prachinburi',
    'ระยอง': 'Rayong',
    'สระแก้ว': 'Sa Kaeo',
    'กาญจนบุรี': 'Kanchanaburi',
    'ตาก': 'Tak',
    'ประจวบคีรีขันธ์': 'Prachuap Khiri Khan',
    'เพชรบุรี': 'Phetchaburi',
    'ราชบุรี': 'Ratchaburi',
    'กระบี่': 'Krabi',
    'ชุมพร': 'Chumphon',
    'ตรัง': 'Trang',
    'นครศรีธรรมราช': 'Nakhon Si Thammarat',
    'นราธิวาส': 'Narathiwat',
    'ปัตตานี': 'Pattani',
    'พังงา': 'Phang Nga',
    'พัทลุง': 'Phatthalung',
    'ภูเก็ต': 'Phuket',
    'ระนอง': 'Ranong',
    'สตูล': 'Satun',
    'สงขลา': 'Songkhla',
    'สุราษฎร์ธานี': 'Surat Thani',
    'ยะลา': 'Yala'})


In [28]:
data_truck_62

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,Sing Buri,1,รถจักรยานยนต์,รถบรรทุก,จอดรถไหล่ทาง
1,01/01/2562,05:00:00,Suphan Buri,2,รถโดยสาร,รถบรรทุก,ขับย้อนศร
2,01/01/2562,12:50:00,Phra Nakhon Si Ayutthaya,4,คู่กรณีมากกว่า 1 คัน,รถบรรทุก,ขับตามหลังในระยะกระชั้นชิด
3,02/01/2562,13:30:00,Loei,3,รถยนต์นั่งส่วนบุคคลไม่เกิน7คน,รถบรรทุก,ขับรถตัดหน้าในระยะกระชั้นชิด
4,02/01/2562,17:00:00,Ranong,1,รถจักรยานยนต์,รถจักรยานยนต์,ขับล้ำเข้าไปในช่องทางจราจรอื่น
...,...,...,...,...,...,...,...
685,29/12/2562,18:02:00,Roi Et,1,รถบรรทุก,รถบรรทุก,ขับล้ำเข้าไปในช่องทางฝั่งตรงข้าม
686,29/12/2562,19:25:00,Sisaket,2,รถจักรยานยนต์,รถบรรทุก,ขับรถตัดหน้าในระยะกระชั้นชิด
687,30/12/2562,11:15:00,Pathum Thani,1,รถบรรทุก,รถบรรทุก,ขับรถตัดหน้าในระยะกระชั้นชิด
688,30/12/2562,11:30:00,Chiang Rai,2,รถบรรทุก,รถบรรทุก,ขับล้ำเข้าไปในช่องทางฝั่งตรงข้าม


In [29]:
#เปลี่ยนสาเหตุเป็นตัวเลขที่กำหนดไว้
data_truck_622 = data_truck_62
datamain62 = pd.DataFrame(data_truck_622)
datamain62['Main cause of accidents'] = datamain62['Main cause of accidents'].apply(lambda x: x.strip())
datamain62['Main cause of accidents'] = datamain62['Main cause of accidents'].replace({'ขับรถเร็ว':'1',
'ขับรถตัดหน้าในระยะกระชั้นชิด':'2',
'ขับตามหลังในระยะกระชั้นชิด':'3',
'หลับใน':'4',
' หลับใน':'4',
'ขับแซงอย่างผิดกฎหมาย/แซงในที่คับขัน':	'5',
'ไม่ชำนาญเส้นทาง':	'6',
'ขาดทักษะที่ถูกต้องในการขับรถ':	'7',
'ขับล้ำเข้าไปในช่องทางจราจรอื่น':	'8',
'ขับล้ำเข้าไปในช่องทางฝั่งตรงข้าม':	'9',
'ฝ่าฝืนเครื่องหมาย/สัญญาณจราจร':	'10',
'เปลี่ยนช่องทางกระทันหัน':	'11',
'เมาสุรา':	'12',
'ใช้สารเสพติดออกฤทธิ์ต่อจิตประสาท':	'13',
'ขาดสมาธิขณะขับรถ':	'14',
'ขับย้อนศร':	'15',
'จอดรถไหล่ทาง':	'16',
'จอดในที่ห้ามจอด/จอดไหล่ทางล้ำ/จอดไม่ให้เครื่องหมายหรือสัญญาณ':	'17',
'ทำกิจกรรมอื่นในขณะขับรถ (ก้มหยิบของ,โทรศัพท์)':	'18',
'สภาพยางชำรุด':	'19',
'ระบบเบรกชำรุด':	'20',
'ระบบคันส่งคันชักชำรุด':	'21',
'ระบบล้อและเพลาชำรุด':	'22',
'ระบบไฟฟ้าชำรุด':	'23',
'ระบบพวงมาลัยชำรุด':	'24',
'อุปกรณ์ส่วนควบอื่นๆชำรุด':	'25',
'สภาพถนนชำรุด':	'26',
'ฝนตกถนนลื่น':	'27',
'ถนนไม่มีไฟฟ้าแสงสว่าง':	'28',
'ถนนมีสิ่งกีดขวางจราจร':	'29',
'ประมาทร่วม':	'30',
'รถอีกคันกระเด็นมาถูกโดยบังเอิญ':	'31',
'ปัญหาสุขภาพผู้ขับรถ':	'32',
'ไม่ระบุ':	'33',
'คนเดินตัดหน้ารถ'	:'34' })

In [30]:
data_truck_622.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,Sing Buri,1,รถจักรยานยนต์,รถบรรทุก,16
1,01/01/2562,05:00:00,Suphan Buri,2,รถโดยสาร,รถบรรทุก,15
2,01/01/2562,12:50:00,Phra Nakhon Si Ayutthaya,4,คู่กรณีมากกว่า 1 คัน,รถบรรทุก,3
3,02/01/2562,13:30:00,Loei,3,รถยนต์นั่งส่วนบุคคลไม่เกิน7คน,รถบรรทุก,2
4,02/01/2562,17:00:00,Ranong,1,รถจักรยานยนต์,รถจักรยานยนต์,8


In [31]:
# เปลี่ยนตัวแปรคู่กรณี (Party) เป็นตัวเลขที่กำหนดไว้
data_truck_6222 = data_truck_622
datapar62 = pd.DataFrame(data_truck_6222)
datapar62['Party'] = datapar62['Party'].apply(lambda x: x.strip())
datapar62['Party'] = datapar62['Party'].replace({'รถยนต์บรรทุกส่วนบุคคล (ปิคอัพ)':	'1',
'รถยนต์นั่งส่วนบุคคลไม่เกิน7คน':	'2',
'รถยนต์นั่งส่วนบุคคลเกิน7คน':	'3',
'รถบรรทุก':	'4',
'รถโดยสาร':	'5',
'รถจักรยานยนต์':	'6',
'รถจักรยาน':	'7',
'รถอื่นๆ':	'8',
'คู่กรณีมากกว่า 1 คัน':	'9',
'เสาไฟฟ้า':	'10',
'ป้าย':'11',
'ต้นไม้':'12',
'ไม่มีคู่กรณี': '13',
'คนเดินเท้า':	'14',
'ไม่ระบุ':	'15' })


In [32]:
data_truck_6222.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,Sing Buri,1,6,รถบรรทุก,16
1,01/01/2562,05:00:00,Suphan Buri,2,5,รถบรรทุก,15
2,01/01/2562,12:50:00,Phra Nakhon Si Ayutthaya,4,9,รถบรรทุก,3
3,02/01/2562,13:30:00,Loei,3,2,รถบรรทุก,2
4,02/01/2562,17:00:00,Ranong,1,6,รถจักรยานยนต์,8


In [33]:
# เปลี่ยนตัวแปรรถต้นเหตุ (Cause) เป็นตัวเลขที่กำหนดไว้
data_truck_62222 = data_truck_6222
datapar62 = pd.DataFrame(data_truck_62222)
datapar62['Cause'] = datapar60['Cause'].apply(lambda x: x.strip())
datapar62['Cause'] = datapar60['Cause'].replace({'รถยนต์บรรทุกส่วนบุคคล (ปิคอัพ)':	'1',
'รถยนต์นั่งส่วนบุคคลไม่เกิน7คน':	'2',
'รถยนต์นั่งส่วนบุคคลเกิน7คน':	'3',
'รถบรรทุก':	'4',
'รถโดยสาร':	'5',
'รถจักรยานยนต์':	'6',
'รถจักรยาน':	'7',
'รถอื่นๆ':	'8',
'คู่กรณีมากกว่า 1 คัน':	'9',
'เสาไฟฟ้า':	'10',
'ป้าย':'11',
'ต้นไม้':'12',
'ไม่มีคู่กรณี': '13',
'คนเดินเท้า':	'14',
'ไม่ระบุ':	'15',})

In [34]:
data_truck_62222.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,12:45:00,Sing Buri,1,6,4,16
1,01/01/2562,05:00:00,Suphan Buri,2,5,4,15
2,01/01/2562,12:50:00,Phra Nakhon Si Ayutthaya,4,9,3,3
3,02/01/2562,13:30:00,Loei,3,2,6,2
4,02/01/2562,17:00:00,Ranong,1,6,3,8


## เปลี่ยนช่วงเวลาให้เป็นตัวเลขที่กำหนดไว้ data truck 2560

In [131]:
k=[]
L=[]
for i in range(len(data_truck_60000)):
  if type(data_truck_60000['Time'][i])==str:
    k.append(i)
  else:
    L.append(i)

In [132]:
data_truck_60000['Time'][476]

datetime.time(21, 21)

In [133]:
data_truck_60000=data_truck_60000.replace(data_truck_60000.iloc[476,1],np.nan)

In [134]:
data_truck_60000.iloc[476,1]

nan

In [136]:
import datetime
data_truck_60000.replace(data_truck_60000.iloc[476,1],np.nan)
def time_in_range(start, end, x):
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

#data_truck_60['Time'] = pd.to_datetime(data_truck_60['Time'], format='%I:%M:%S' or '%I:%M:%S %p', errors='coerce').dt.time
time_labels = []
how = []

start_1 = datetime.time(16, 1, 0)
end_1 = datetime.time(20, 0, 59)

start_2 = datetime.time(20, 1, 0)
end_2 = datetime.time(0, 0, 59)

start_3 = datetime.time(0, 1, 0)
end_3 = datetime.time(4, 0, 59)

start_4 = datetime.time(4, 1, 0)
end_4 = datetime.time(8, 0, 59)

start_5 = datetime.time(8, 1, 0)
end_5 = datetime.time(12, 0, 59)

start_6 = datetime.time(12, 1, 0)
end_6 = datetime.time(16, 0, 0)

filtered_data = data_truck_60.dropna(subset=['Time'])

for i in range(len(filtered_data['Time'])):
    if time_in_range(start_1, end_1, filtered_data['Time'].iloc[i])== True:
        time_labels.append(1)
    elif time_in_range(start_2, end_2, filtered_data['Time'].iloc[i])== True:
        time_labels.append(2)
    elif time_in_range(start_3, end_3, filtered_data['Time'].iloc[i])== True:
        time_labels.append(3)
    elif time_in_range(start_4, end_4, filtered_data['Time'].iloc[i])== True:
        time_labels.append(4)
    elif time_in_range(start_5, end_5, filtered_data['Time'].iloc[i])== True:
        time_labels.append(5)
    elif time_in_range(start_6, end_6, filtered_data['Time'].iloc[i])== True:
        time_labels.append(6)
    else:
        how.append(i)

print(time_labels)

[5, 4, 2, 2, 4, 3, 5, 4, 6, 1, 2, 4, 5, 1, 3, 4, 6, 6, 3, 2, 5, 1, 4, 3, 4, 2, 4, 4, 4, 4, 1, 1, 2, 2, 5, 1, 3, 2, 6, 2, 5, 4, 2, 6, 3, 5, 1, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3, 4, 5, 4, 4, 6, 4, 4, 4, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 3, 5, 4, 6, 4, 5, 4, 4, 5, 6, 1, 2, 2, 2, 2, 6, 5, 1, 5, 1, 2, 3, 2, 4, 4, 6, 1, 2, 1, 1, 2, 3, 5, 1, 2, 2, 4, 5, 1, 2, 4, 1, 1, 5, 4, 4, 5, 6, 5, 1, 2, 3, 5, 5, 1, 2, 4, 5, 5, 5, 1, 2, 5, 6, 6, 6, 1, 2, 5, 5, 6, 5, 6, 6, 5, 5, 1, 1, 6, 2, 5, 6, 6, 1, 2, 3, 4, 1, 6, 4, 5, 6, 6, 2, 6, 5, 2, 6, 4, 1, 5, 2, 2, 5, 2, 2, 3, 4, 6, 2, 5, 1, 2, 4, 3, 5, 4, 3, 4, 4, 5, 5, 3, 5, 6, 5, 6, 5, 3, 5, 4, 4, 5, 5, 5, 4, 5, 4, 4, 4, 5, 5, 5, 4, 5, 4, 3, 6, 3, 5, 4, 5, 3, 4, 5, 5, 3, 4, 5, 4, 5, 5, 4, 3, 5, 5, 4, 5, 5, 3, 5, 5, 5, 4, 5, 4, 4, 4, 4, 4, 4, 3, 5, 4, 4, 5, 4, 5, 4, 4, 4, 4, 5, 5, 4, 5, 6, 5, 6, 5, 3, 4, 4, 5, 4, 5, 3, 5, 4, 4, 4, 4, 1, 3, 4, 5, 2, 3, 6, 6, 4, 5, 3, 4, 2, 2, 4, 4, 5, 1, 1, 2, 4, 4, 6, 6, 6, 2, 6, 5, 5, 5, 6, 1, 2, 4, 5, 6, 3, 3, 3, 4, 

In [137]:
time_labels = pd.DataFrame({
    'Time': [5, 4, 4, 3, 5, 4, 4, 5, 3, 4, 3, 5, 4, 3, 4, 4, 4, 4, 4, 5, 3, 5, 4, 3, 3, 5, 4, 4, 4, 5, 5, 5, 5, 2, 2, 5, 4, 3, 4, 4, 4, 3, 4, 4, 4, 5, 5, 4, 5, 4, 5, 3, 5, 4, 4, 2, 4, 4, 2, 5, 5, 3, 4, 4, 3, 5, 4, 5, 4, 5, 4, 4, 5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 3, 5, 3, 4, 4, 5, 3, 5, 4, 5, 5, 3, 4, 5, 4, 3, 4, 3, 4, 4, 3, 5, 5, 3, 5, 3, 5, 4, 4, 2, 4, 4, 4, 4, 2, 5, 4, 4, 3, 3, 5, 4, 5, 3, 4, 5, 5, 3, 4, 5, 4, 2, 5, 4, 3, 5, 4, 5, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 5, 4, 4, 4, 4, 4, 2, 4, 5, 5, 3, 3, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 5, 3, 3, 4, 5, 3, 4, 4, 4, 2, 4, 4, 3, 3, 5, 5, 5, 4, 5, 3, 3, 3, 4, 5, 4, 3, 4, 3, 2, 4, 5, 4, 4, 3, 5, 3, 3, 3, 5, 3, 4, 4, 3, 4, 4, 5, 3, 5, 5, 3, 3, 4, 2, 4, 5, 5, 3, 3, 5, 4, 4, 4, 5, 3, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 5, 5, 4, 3, 4, 3, 4, 4, 5, 5, 3, 3, 4, 3, 4, 5, 3, 3, 4, 4, 4, 5, 3, 2, 5, 4, 5, 5, 3, 5, 4, 4, 3, 3, 5, 4, 3, 4, 5, 3, 3, 4, 5, 4, 5, 3, 4, 4, 3, 5, 5, 5, 5, 3, 3, 3, 4, 5, 4, 5, 4, 4, 5, 5, 3, 3, 3, 3, 4, 5, 5, 3, 4, 5, 4, 3, 5, 3, 3, 4, 4, 3, 4, 3, 3, 3, 4, 4, 3, 3, 5, 4, 5, 3, 5, 4, 5, 2, 4, 4, 5, 2, 3, 4, 3, 4, 5, 3, 4, 4, 5, 5, 4, 3, 4, 5, 3, 4, 3, 4, 4, 3, 3, 4, 4, 5, 4, 4, 5, 4, 3, 4, 5, 3, 5, 3, 4, 3, 3, 3, 4, 4, 4, 4, 5, 4, 4, 3, 3, 4, 4, 5, 4, 5, 5, 3, 4, 4, 3, 4, 5, 4, 4, 4, 4, 5, 4, 4, 5, 5, 4, 4, 5, 3, 3, 3, 3, 4, 4, 4, 4, 3, 5, 5, 3, 4, 4, 5, 3, 5, 5, 4, 5, 3, 3, 4, 4, 4, 5, 5, 5, 4, 4, 4, 5, 5, 3, 3, 5, 5, 5, 4, 5, 5, 4, 5, 4, 4, 2, 3, 4, 5, 5],})

In [138]:
data_truck_60000['Time'] = time_labels
print(data_truck_60000)

           Date  Time             Province  Loss Party Cause  \
0    02/01/2560   5.0            Khon Kaen     3    P8    C4   
1    04/01/2560   4.0                Phrae     2   P13    C4   
2    04/01/2560   4.0             Saraburi     2    P3    C3   
3    04/01/2560   3.0          Phetchaburi     2    P6    C6   
4    05/01/2560   5.0           Chiang Rai     2    P3    C3   
..          ...   ...                  ...   ...   ...   ...   
878  29/12/2560   NaN         Samut Prakan     3    P6    C6   
879  30/12/2560   NaN             Chonburi     2    P6    C6   
880  30/12/2560   NaN              Lampang     1    P6    C6   
881  31/12/2560   NaN  Prachuap Khiri Khan     1    P6    C6   
882  31/12/2560   NaN                 Trat     1   P14   C14   

    Main cause of accidents  
0                        M4  
1                        M4  
2                        M5  
3                        M5  
4                        M1  
..                      ...  
878                  

In [139]:
data_truck_60000

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5.0,Khon Kaen,3,P8,C4,M4
1,04/01/2560,4.0,Phrae,2,P13,C4,M4
2,04/01/2560,4.0,Saraburi,2,P3,C3,M5
3,04/01/2560,3.0,Phetchaburi,2,P6,C6,M5
4,05/01/2560,5.0,Chiang Rai,2,P3,C3,M1
...,...,...,...,...,...,...,...
878,29/12/2560,NaN,Samut Prakan,3,P6,C6,M3
879,30/12/2560,NaN,Chonburi,2,P6,C6,M2
880,30/12/2560,NaN,Lampang,1,P6,C6,M2
881,31/12/2560,NaN,Prachuap Khiri Khan,1,P6,C6,M2


In [146]:
# ลบจุดทศนิยม
data_truck_600000['Time'] = data_truck_60000['Time'].astype(str)
data_truck_60000['Time'] = data_truck_60000['Time'].apply(lambda x: x.strip("0. "))

In [147]:
data_truck_600000

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5,Khon Kaen,3,P8,C4,M4
1,04/01/2560,4,Phrae,2,P13,C4,M4
2,04/01/2560,4,Saraburi,2,P3,C3,M5
3,04/01/2560,3,Phetchaburi,2,P6,C6,M5
4,05/01/2560,5,Chiang Rai,2,P3,C3,M1
...,...,...,...,...,...,...,...
878,29/12/2560,nan,Samut Prakan,3,P6,C6,M3
879,30/12/2560,nan,Chonburi,2,P6,C6,M2
880,30/12/2560,nan,Lampang,1,P6,C6,M2
881,31/12/2560,nan,Prachuap Khiri Khan,1,P6,C6,M2


In [150]:
# เปลี่ยนเวลาจากตัวเลขให้เป็นช่วง

data_truck_6000001 = data_truck_600000
data_truck_600000 = pd.DataFrame(data_truck_6000001)
data_truck_600000['Time'] = data_truck_600000['Time'].apply(lambda x: x.strip())
data_truck_600000['Time'] = data_truck_600000['Time'].replace({'1': '16.01 - 20.00',
'2':'20.01 - 00.00',
'3':'00.01 - 04.00',
'4': '04.01 - 08.00 ',
'5': '08.01 - 12.00',
'6':'12.01 - 16.00'})

In [161]:
data_truck_6000001.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,08.01 - 12.00,Khon Kaen,3,P8,C4,M4
1,04/01/2560,04.01 - 08.00,Phrae,2,P13,C4,M4
2,04/01/2560,04.01 - 08.00,Saraburi,2,P3,C3,M5
3,04/01/2560,00.01 - 04.00,Phetchaburi,2,P6,C6,M5
4,05/01/2560,08.01 - 12.00,Chiang Rai,2,P3,C3,M1


## เปลี่ยนช่วงเวลาให้เป็นตัวเลขที่กำหนดไว้ data truck 2562

In [152]:
k=[]
L=[]
for i in range(len(data_truck_62222)):
  if type(data_truck_62222['Time'][i])==str:
    k.append(i)
  else:
    L.append(i)

In [153]:
import datetime

def time_in_range(start, end, x):
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end
time_labels = []
how = []
data_truck_62222['Time'] = pd.to_datetime(data_truck_62222['Time'], format='%I:%M:%S' or '%I:%M:%S %p', errors='coerce').dt.time


start_1 = datetime.time(16, 1, 0)
end_1 = datetime.time(20, 0, 59)

start_2 = datetime.time(20, 1, 0)
end_2 = datetime.time(0, 0, 59)

start_3 = datetime.time(0, 1, 0)
end_3 = datetime.time(4, 0, 59)

start_4 = datetime.time(4, 1, 0)
end_4 = datetime.time(8, 0, 59)

start_5 = datetime.time(8, 1, 0)
end_5 = datetime.time(12, 0, 59)

start_6 = datetime.time(12, 1, 0)
end_6 = datetime.time(16, 0, 0)
filtered_data = data_truck_62.dropna(subset=['Time'])

for i in range(len(filtered_data['Time'])):
    if time_in_range(start_1, end_1, filtered_data['Time'].iloc[i])== True:
        time_labels.append(1)
    elif time_in_range(start_2, end_2, filtered_data['Time'].iloc[i])== True:
        time_labels.append(2)
    elif time_in_range(start_3, end_3, filtered_data['Time'].iloc[i])== True:
        time_labels.append(3)
    elif time_in_range(start_4, end_4, filtered_data['Time'].iloc[i])== True:
        time_labels.append(4)
    elif time_in_range(start_5, end_5, filtered_data['Time'].iloc[i])== True:
        time_labels.append(5)
    elif time_in_range(start_6, end_6, filtered_data['Time'].iloc[i])== True:
        time_labels.append(6)
    else:
        how.append(i)

# Print the time labels for verification
print(time_labels)

[]


In [154]:
time_labels2 = pd.DataFrame({
    'Time': [3, 4, 6, 6, 1, 5, 2, 3, 1, 3, 5, 5, 6, 5, 6, 3, 4, 5, 4, 5, 3, 4, 5, 6, 1, 2, 3, 5, 6, 1, 1, 2, 3, 4, 4, 5, 3, 3, 5, 4, 2, 3, 4, 5, 2, 4, 4, 5, 6, 3, 3, 4, 6, 6, 6, 4, 3, 1, 1, 1, 1, 2, 4, 4, 2, 3, 5, 1, 5, 5, 5, 1, 1, 4, 5, 2, 2, 6, 1, 2, 4, 6, 1, 2, 4, 3, 6, 2, 2, 3, 4, 4, 3, 3, 5, 6, 4, 2, 4, 5, 3, 4, 2, 3, 5, 1, 6, 3, 3, 6, 1, 1, 4, 5, 6, 3, 4, 2, 2, 2, 3, 3, 2, 3, 4, 6, 3, 4, 6, 1, 3, 4, 4, 5, 6, 1, 5, 6, 1, 3, 4, 4, 4, 1, 1, 4, 4, 1, 6, 1, 5, 2, 2, 2, 3, 3, 6, 5, 5, 6, 3, 3, 5, 3, 1, 2, 1, 3, 3, 4, 4, 4, 1, 3, 4, 3, 3, 4, 3, 5, 3, 3, 6, 2, 5, 5, 5, 6, 4, 4, 5, 6, 4, 5, 6, 3, 4, 4, 5, 4, 1, 3, 3, 1, 6, 6, 2, 4, 5, 6, 5, 6, 6, 2, 4, 2, 4, 4, 6, 1, 6, 2, 2, 1, 3, 3, 4, 5, 5, 2, 2, 3, 3, 3, 4, 5, 1, 1, 1, 2, 4, 5, 5, 1, 2, 2, 3, 3, 2, 6, 5, 5, 2, 4, 5, 1, 4, 4, 4, 4, 4, 5, 3, 6, 6, 4, 6, 6, 3, 6, 1, 5, 6, 6, 4, 4, 4, 4, 5, 2, 2, 5, 6, 4, 6, 4, 3, 3, 6, 5, 6, 3, 4, 4, 4, 3, 5, 3, 4, 4, 6, 4, 5, 1, 3, 4, 2, 5, 4, 6, 4, 5, 2, 3, 3, 4, 6, 1, 5, 1, 1, 3, 4, 1, 6, 5, 5, 3, 5, 1, 1, 1, 2, 2, 4, 1, 1, 2, 6, 3, 5, 1, 5, 1, 3, 4, 5, 1, 4, 5, 5, 4, 4, 5, 5, 6, 4, 5, 3, 4, 4, 5, 6, 2, 3, 5, 1, 2, 2, 1, 2, 2, 4, 5, 5, 6, 6, 2, 5, 2, 1, 5, 3, 3, 2, 4, 5, 6, 6, 2, 3, 6, 1, 4, 4, 6, 4, 2, 5, 1, 6, 3, 5, 4, 2, 2, 6, 3, 6, 2, 1, 4, 5, 6, 4, 5, 4, 5, 3, 5, 6, 1, 1, 6, 2, 3, 5, 6, 6, 6, 3, 3, 4, 5, 3, 4, 5, 6, 6, 6, 4, 2, 4, 4, 3, 4, 1, 3, 6, 1, 2, 6, 2, 3, 5, 6, 5, 4, 4, 4, 4, 4, 1, 6, 3, 3, 6, 1, 4, 5, 2, 3, 2, 6, 1, 3, 4, 6, 5, 6, 2, 3, 4, 5, 2, 5, 1, 2, 6, 3, 2, 3, 3, 1, 5, 1, 3, 5, 6, 4, 4, 4, 4, 3, 2, 4, 2, 3, 4, 5, 1, 2, 1, 6, 2, 3, 6, 4, 5, 4, 3, 4, 2, 3, 4, 6, 3, 4, 1, 3, 3, 6, 1, 2, 2, 4, 6, 4, 2, 3, 4, 6, 4, 1, 5, 4, 4, 4, 1, 2, 4, 5, 2, 3, 4, 6, 1, 5, 5, 5, 6, 2, 4, 4, 5, 3, 2, 4, 5, 4, 5, 4, 1, 2, 4, 5, 1, 4, 4, 6, 1, 1, 2, 2, 2, 1, 4, 5, 3, 4, 1, 5, 6, 3, 2, 3, 1, 4, 6, 1, 1, 1, 5, 6, 5, 3, 3, 5, 6, 1, 4, 4, 1, 2, 2, 3, 4, 1, 3, 4, 3, 4, 5, 5, 6, 1, 2, 2, 5, 5, 1, 2, 2, 4, 5, 3, 5, 3, 4, 3, 5, 4, 5, 1, 4, 4, 6, 4, 2, 2, 6, 1, 3, 5, 2, 3, 3, 6, 2, 3, 6, 2, 3, 4, 3, 1, 3, 3, 6, 2, 2, 4, 2, 6, 3, 1, 1, 4, 4, 4, 6, 6, 6, 4, 5, 1, 1, 5, 5, 1],})

In [155]:
data_truck_62222['Time'] = time_labels2
print(data_truck_62222)

           Date  Time                  Province  Loss Party Cause  \
0    01/01/2562     3                 Sing Buri     1     6     4   
1    01/01/2562     4               Suphan Buri     2     5     4   
2    01/01/2562     6  Phra Nakhon Si Ayutthaya     4     9     3   
3    02/01/2562     6                      Loei     3     2     6   
4    02/01/2562     1                    Ranong     1     6     3   
..          ...   ...                       ...   ...   ...   ...   
685  29/12/2562     1                    Roi Et     1     4     4   
686  29/12/2562     1                   Sisaket     2     6     4   
687  30/12/2562     5              Pathum Thani     1     4     4   
688  30/12/2562     5                Chiang Rai     2     4     4   
689  30/12/2562     1                       Nan     1     4     4   

    Main cause of accidents  
0                        16  
1                        15  
2                         3  
3                         2  
4                    

In [157]:
# ลบจุดทศนิยม
data_truck_62222['Time'] = data_truck_62222['Time'].astype(str)
data_truck_62222['Time'] = data_truck_62222['Time'].apply(lambda x: x.strip("0. "))

In [158]:
data_truck_622222 = data_truck_62222
data_truck_62222 = pd.DataFrame(data_truck_62222)
data_truck_62222['Time'] = data_truck_62222['Time'].apply(lambda x: x.strip())
data_truck_62222['Time'] = data_truck_62222['Time'].replace({'1': '16.01 - 20.00',
'2':'20.01 - 00.00',
'3':'00.01 - 04.00',
'4': '04.01 - 08.00 ',
'5': '08.01 - 12.00',
'6':'12.01 - 16.00'})

In [160]:
data_truck_622222.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,00.01 - 04.00,Sing Buri,1,6,4,16
1,01/01/2562,04.01 - 08.00,Suphan Buri,2,5,4,15
2,01/01/2562,12.01 - 16.00,Phra Nakhon Si Ayutthaya,4,9,3,3
3,02/01/2562,12.01 - 16.00,Loei,3,2,6,2
4,02/01/2562,16.01 - 20.00,Ranong,1,6,3,8


# Data ที่ได้

### Data truck 2560

In [46]:
data_truck_60000

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5.0,Khon Kaen,3,8,4,4
1,04/01/2560,4.0,Phrae,2,13,4,4
2,04/01/2560,4.0,Saraburi,2,3,3,5
3,04/01/2560,3.0,Phetchaburi,2,6,6,5
4,05/01/2560,5.0,Chiang Rai,2,3,3,1
...,...,...,...,...,...,...,...
878,29/12/2560,NaN,Samut Prakan,3,6,6,3
879,30/12/2560,NaN,Chonburi,2,6,6,2
880,30/12/2560,NaN,Lampang,1,6,6,2
881,31/12/2560,NaN,Prachuap Khiri Khan,1,6,6,2


In [162]:
# ลบจุดทศนิยม
data_truck_60000['Time'] = data_truck_60000['Time'].astype(str)
data_truck_60000['Time'] = data_truck_60000['Time'].apply(lambda x: x.strip("0. "))

In [166]:
data_truck_600001 = data_truck_60000
data_truck_60000 = pd.DataFrame(data_truck_60000)
data_truck_60000['Time'] = data_truck_60000['Time'].apply(lambda x: x.strip())
data_truck_60000['Time'] = data_truck_60000['Time'].replace({'1': '16.01 - 20.00',
'2':'20.01 - 00.00',
'3':'00.01 - 04.00',
'4': '04.01 - 08.00 ',
'5': '08.01 - 12.00',
'6':'12.01 - 16.00'})

In [167]:
data_truck_600001.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,8.01 - 12,Khon Kaen,3,P8,C4,M4
1,04/01/2560,4.01 - 08,Phrae,2,P13,C4,M4
2,04/01/2560,4.01 - 08,Saraburi,2,P3,C3,M5
3,04/01/2560,1 - 04,Phetchaburi,2,P6,C6,M5
4,05/01/2560,8.01 - 12,Chiang Rai,2,P3,C3,M1


### Data truck 2561

In [47]:
data_truck_61.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2561,2.0,Samut Sakhon,1,6,1,16
1,01/01/2561,2.0,Rayong,1,6,1,16
2,02/01/2561,5.0,Samut Sakhon,3,1,1,19
3,03/01/2561,6.0,Surat Thani,2,9,1,6
4,03/01/2561,1.0,Rayong,1,6,6,1


### Data truck 2562

In [48]:
data_truck_62222.head()

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,01/01/2562,3,Sing Buri,1,6,4,16
1,01/01/2562,4,Suphan Buri,2,5,4,15
2,01/01/2562,6,Phra Nakhon Si Ayutthaya,4,9,3,3
3,02/01/2562,6,Loei,3,2,6,2
4,02/01/2562,1,Ranong,1,6,3,8


# Check Missing values

In [49]:
data_truck_60000.isnull().any()

Date                       False
Time                        True
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
dtype: bool

In [50]:
data_truck_61.isnull().any()

Date                       False
Time                        True
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
dtype: bool

In [51]:
data_truck_62222.isnull().any()

Date                       False
Time                       False
Province                   False
Loss                       False
Party                      False
Cause                      False
Main cause of accidents    False
dtype: bool

# Concat Data

In [52]:
data_concat = pd.concat([data_truck_60000,data_truck_61,data_truck_62222])
data_concat

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5.0,Khon Kaen,3,8,4,4
1,04/01/2560,4.0,Phrae,2,13,4,4
2,04/01/2560,4.0,Saraburi,2,3,3,5
3,04/01/2560,3.0,Phetchaburi,2,6,6,5
4,05/01/2560,5.0,Chiang Rai,2,3,3,1
...,...,...,...,...,...,...,...
685,29/12/2562,1.0,Roi Et,1,4,4,9
686,29/12/2562,1.0,Sisaket,2,6,4,2
687,30/12/2562,5.0,Pathum Thani,1,4,4,2
688,30/12/2562,5.0,Chiang Rai,2,4,4,9


In [53]:
data_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2431 entries, 0 to 689
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date                     2431 non-null   object 
 1   Time                     2021 non-null   float64
 2   Province                 2431 non-null   object 
 3   Loss                     2431 non-null   int64  
 4   Party                    2431 non-null   object 
 5   Cause                    2431 non-null   object 
 6   Main cause of accidents  2431 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 151.9+ KB


In [54]:
set(data_concat['Party'])

{1,
 '1',
 '13',
 13,
 14,
 '14',
 15,
 '15',
 2,
 '2',
 3,
 '3',
 4,
 '4',
 5,
 '5',
 6,
 '6',
 '7',
 7,
 8,
 '8',
 9,
 '9'}

# ลองเปลี่ยนไทป์ data

In [128]:
import pandas as pd


data_concat = {'Party': [1,'1','13',13,14,'14',15,'15',2,'2',3,'3',4,'4',5,'5',6,'6','7',7,8,'8',9,'9']}
data_concat['Party'] = data_concat['Party'].astype(str)

AttributeError: 'set' object has no attribute 'astype'

In [126]:
set(data_concat['Party'])

{'1', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9'}

In [55]:
data_concat[(data_concat['Cause'] == '4' ) | (data_concat['Party'] == '4')]

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents
0,02/01/2560,5.0,Khon Kaen,3,8,4,4
1,04/01/2560,4.0,Phrae,2,13,4,4
8,09/01/2560,3.0,Phetchaburi,4,4,15,33
9,09/01/2560,4.0,Nakhon Sawan,4,4,4,2
10,09/01/2560,3.0,Pathum Thani,1,4,4,2
...,...,...,...,...,...,...,...
685,29/12/2562,1.0,Roi Et,1,4,4,9
686,29/12/2562,1.0,Sisaket,2,6,4,2
687,30/12/2562,5.0,Pathum Thani,1,4,4,2
688,30/12/2562,5.0,Chiang Rai,2,4,4,9


# guide

In [56]:
test_df = data_concat.copy()

In [57]:
test_df['for_association'] = test_df[['Party','Cause']].apply(list, axis=1)

In [58]:
test_df

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,for_association
0,02/01/2560,5.0,Khon Kaen,3,8,4,4,"[8, 4]"
1,04/01/2560,4.0,Phrae,2,13,4,4,"[13, 4]"
2,04/01/2560,4.0,Saraburi,2,3,3,5,"[3, 3]"
3,04/01/2560,3.0,Phetchaburi,2,6,6,5,"[6, 6]"
4,05/01/2560,5.0,Chiang Rai,2,3,3,1,"[3, 3]"
...,...,...,...,...,...,...,...,...
685,29/12/2562,1.0,Roi Et,1,4,4,9,"[4, 4]"
686,29/12/2562,1.0,Sisaket,2,6,4,2,"[6, 4]"
687,30/12/2562,5.0,Pathum Thani,1,4,4,2,"[4, 4]"
688,30/12/2562,5.0,Chiang Rai,2,4,4,9,"[4, 4]"


In [59]:
test_df[test_df['Time'] == 1]

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,for_association
4,03/01/2561,1.0,Rayong,1,6,6,1,"[6, 6]"
13,06/01/2561,1.0,Phetchabun,1,13,1,6,"[13, 1]"
16,07/01/2561,1.0,Phrae,4,4,1,3,"[4, 1]"
24,09/01/2561,1.0,Phrae,1,6,6,30,"[6, 6]"
27,10/01/2561,1.0,Phrae,1,6,6,5,"[6, 6]"
...,...,...,...,...,...,...,...,...
675,26/12/2562,1.0,Prachuap Khiri Khan,2,4,4,3,"[4, 4]"
676,26/12/2562,1.0,Nakhon Ratchasima,4,9,5,3,"[9, 5]"
685,29/12/2562,1.0,Roi Et,1,4,4,9,"[4, 4]"
686,29/12/2562,1.0,Sisaket,2,6,4,2,"[6, 4]"


In [60]:
test_split = test_df.copy()

In [61]:
test_split[['day', 'month', 'year']] = test_split['Date'].str.split('/', n=2, expand = True)

In [62]:
test_split

,Date,Time,Province,Loss,Party,Cause,Main cause of accidents,for_association,day,month,year
0,02/01/2560,5.0,Khon Kaen,3,8,4,4,"[8, 4]",02,01,2560
1,04/01/2560,4.0,Phrae,2,13,4,4,"[13, 4]",04,01,2560
2,04/01/2560,4.0,Saraburi,2,3,3,5,"[3, 3]",04,01,2560
3,04/01/2560,3.0,Phetchaburi,2,6,6,5,"[6, 6]",04,01,2560
4,05/01/2560,5.0,Chiang Rai,2,3,3,1,"[3, 3]",05,01,2560
...,...,...,...,...,...,...,...,...,...,...,...
685,29/12/2562,1.0,Roi Et,1,4,4,9,"[4, 4]",29,12,2562
686,29/12/2562,1.0,Sisaket,2,6,4,2,"[6, 4]",29,12,2562
687,30/12/2562,5.0,Pathum Thani,1,4,4,2,"[4, 4]",30,12,2562
688,30/12/2562,5.0,Chiang Rai,2,4,4,9,"[4, 4]",30,12,2562
